## SIMPLEST SUPPLY/DEMAND DEMO

- Base Model: 
   * 2 techs competing for energy source(lamp and heater), for 2 sources  (electricity and gas)
   * fix supply capital cost to only initial factory cost, not changing costs like labour
   * fix freq parameter (energy saving weight) to 1 (check: no freq for supply??)
   * technologies don't die or become obsolete or change
   * full quantity does not change
   * capital cost does not change
   


TO DO: Find out how the functional form (or at least slope as linear) of demand is known. I was under the impression that only the supply function was fully unknown.

Might be advantageous to build an iterator instead of while loop to find equilibrium

In [1]:
# !jupyter nbconvert --to script supply_demand_new.ipynb

In [2]:
%matplotlib inline

import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt

from helper.utils import * # for functions `process_subtypes`, `filter_fun`, `init_logger`

# consider using OrderedDict instead of dict

In [3]:
# Start a log file (from utils module)
init_logger(filename="supply_demand")



### Fake Data Setup:

This data is structured as if it were the result of passing through the full model until the leaf and 

Note: I introduced a rate parameter to indicate the consumption rate of a tech unit

In [4]:
'''
THINK OF the problem of adjusting fuel quantity when there is demand from different 'cells',
and at different levels
'''

data = {"demand_step":[{"kind": "furnace",
                        "priceUnits": "$",
                        "unit": "GJ",
                        "total_demand": 10000,
                        "leaf": True, # might need to change the 'leaf' paradigm (end use? unique key?)
                        "subtypes": {"furnace1": {"fuel": "electricity",
                                                  "demand": None,
                                                  "price": 200,
                                                  "cap_cost": 10},
                                     "furnace2": {"fuel": "electricity",
                                                  "demand": None,
                                                  "price": 200,
                                                  "cap_cost": 15},
                                     "furnace3": {"fuel": "gas",
                                                  "demand": None,
                                                  "price": 150,
                                                  "cap_cost": 30},
                                     "furnace4": {"fuel": "gas",
                                                  "demand": None,
                                                  "price": 150,
                                                  "cap_cost": 50}
                                     } 
                        },
                        {"kind": "hot_water",
#                          "end_use": True, #?
                         "priceUnits": "$",
                         "unit": "GJ",
                         "total_demand": 9000,
                         "leaf": True,
                         "subtypes": {"hot_water1": {"fuel": "electricity",
                                                     "demand": None, 
                                                     "price": 2,
                                                     "cap_cost": 5},
                                      "hot_water2": {"fuel": "electricity",
                                                     "demand": None,
                                                     "price": 2,
                                                     "cap_cost": 2},
                                      "hot_water3": {"fuel": "gas",
                                                     "demand": None,
                                                     "price": 1.5,
                                                     "cap_cost": 15}
                                      }
                        }],

          "supply_step":[{"kind": "electricity",
                          "leaf": True,
                          "supply": 5000,
                          "subtypes": {"wind": {"price": 110,
                                                "cap_cost": 2},
                                       "hydro": {"price": 100,
                                                 "cap_cost": 5},
                                       "rocks": {"price": 80,
                                                 "cap_cost": 10}
                                       } 
                          },
                          {"kind": "gas",
                           "leaf": True,
                           "supply": 3000,
                           "subtypes": {"gas1": {"price": 150,
                                                 "cap_cost": 10},
                                        "gas2": {"price": 120,
                                                 "cap_cost": 0}
                                        }
                          }]
       }   

#### Example for selecting certain values that meet a requirement with the `process_subtypes` function defined in `utils.py`:

Here showing values in all technologies that use electricity.

In [5]:
# by_elec = filter_obs(data, fuel_type = "electricity", val = "fuel", verbose=True)

#### Define classes for Supply and Demand:

In [6]:
class Demand:
    '''
    
    - **kwargs: dict of named arguments in "argname": value pairs to specify used economic indicators (GDP, population, etc)
    '''

    def __init__(self, data, v=10, return_all_info=False, **kwargs):
        
        self.dtechs = data["demand_step"]
        self.dfuels = data["supply_step"]
        
        lccs = self.lcc_over_tech()
        
        if return_all_info:
            tech_demand, tech_mshares = self.quantity_demanded(lccs, v=v, return_all_info=True)
            fuel_demand = self.demand_by_fuel(tech_demand)
            self.fuel_demand, self.tech_demand, self.tech_mshares = fuel_demand, tech_demand, tech_mshares
        else:
            tech_demand = self.quantity_demanded(lccs, v=v, return_all_info=False)
            fuel_demand = self.demand_by_fuel(tech_demand)
            self.fuel_demand = fuel_demand
        
        
    ## Methods for demand side calculations 
    def life_cycle_cost(self, last_price, cap_cost=0):
        lcc = (last_price + cap_cost)
        return lcc
    
    def market_share(self, lcc, techname, v=10):
        # lcc is a dict
        sum_v = sum(lcc[item]**(-1.0*v) for item in lcc)
        ms = (lcc[techname]**(-1.0*v))/sum_v
        return ms
        
    def get_quantity(self, ms, fuel_quantity):
        demand = ms * fuel_quantity
        return demand
    
    def lcc_over_tech(self, keytech = "kind", keysubtech = "subtypes",
                            keyprice = "price", keycapcost = "cap_cost"):
        # TODO: use some kwargs in lcc method in case there are arbitrary numbers of params
        lccs = {}
        # techdict is the entire subtree for the kth competing tech (demand side)
        for techdict in self.dtechs:
            # get the tech name (key) to use as a label for lcc
            tech_k = techdict[keytech]
            lccs[tech_k] = {}
            # fetch all attributes/values for the (sub)tech that demands a fuel
            # by `subtech` I mean e.g.: tech: furnace, subtech: type of furnace
            subtechs = techdict[keysubtech]            
            # loop over `subtechs`, get subtech name (key) and vals (all attributes associated w/ subtech)
            for subtech, vals in subtechs.items():
                # unpack components needed for lcc
                price = vals[keyprice]
                cc = vals[keycapcost]
                lccs[tech_k][subtech] = self.life_cycle_cost(last_price=price, cap_cost=cc) # kwargs here
        return lccs        
                
                
    def quantity_demanded(self, lccs, v=10, return_all_info=True, keytech="kind", 
                                keysubtech="subtypes", keyquant="total_demand"):
        mshares, quants = {}, {}

        # techdict is the entire subtree for the kth competing tech (demand side)            
        for techdict in self.dtechs:
            # fetch all attributes/values for the (sub)tech that demands a fuel
            # by `subtech` I mean e.g.: tech: furnace, subtech: type of furnace
            subtechs = techdict[keysubtech]
            # loop over `subtechs`, get subtech name (key) and vals (all attributes associated w/ subtech)                
                
            for tech, vals in subtechs.items():
                mshares[tech] = self.market_share(lccs[techdict[keytech]], tech, v=v)
                quants[tech] = self.get_quantity(mshares[tech], techdict[keyquant])     
                    
        if return_all_info:
            return quants, mshares
        else:
            return quants
        
    def demand_by_fuel(self, quants, keyfuel="kind", keytechfuel="fuel", keysubtech="subtypes"):
        by_fuel, sum_quants = {}, {}

        # get only numbers for a given fuel, in both techs
        for fkind in self.dfuels:
            cur_fuel = fkind[keyfuel]
            by_fuel[cur_fuel] = filter_obs(data, fuel_type = cur_fuel, val = "fuel", verbose=False)
            sum_quants[cur_fuel] = sum(quants[subtype] for subtype in by_fuel[cur_fuel].keys())
        return sum_quants    
                    



In [7]:
class Supply:
    def __init__(self, data, v=10, return_all_info=True, **kwargs):
        
        self.sfuels = data["supply_step"]
        lccs = self.lcc_over_production()
        
        if return_all_info:
            self.tot_price, self.price, self.sup_mshares = self.get_fuelprice(lccs, v)
        else:
            self.tot_price = self.get_fuelprice(lccs, v)            
            
            
    ## Methods for calculating demand side 
    def life_cycle_cost(self, last_price, cap_cost=0):
        lcc = (last_price + cap_cost)
        return lcc
        
    def market_share(self, lcc, prodname, v=10):
        # lcc is a dict
        sum_v = sum(lcc[item]**(-1.0*v) for item in lcc)
        ms = (lcc[prodname]**(-1.0*v))/sum_v
        return ms
        
    def get_price(self, ms, lcc):
        price = ms*lcc       
        return price
                
        
        
    def lcc_over_production(self, keyfuel = "kind", keyprod = "subtypes",
                                  keyprice = "price", keycapcost = "cap_cost"):  
        lccs = {}        
        for fueldict in self.sfuels:
            # loop through each fuel, fueldict is entire subtree below each type of fuel
            fueltype = fueldict[keyfuel]
            lccs[fueltype] = {}
            
            for prod, vals in fueldict[keyprod].items():
                # loop through each production type (say 'hydro' if fuel is 'electricity')
                lccs[fueltype][prod] = self.life_cycle_cost(last_price=vals[keyprice], cap_cost=vals[keycapcost])
        return lccs    
    
    def get_fuelprice(self, lccs, v = 10, keyfuel = "kind", keyprod = "subtypes",
                           keyprice = "price", keycapcost = "cap_cost",
                           return_all_info = True): 
        # distinguish the 2 types of lccs (one for added, one for price)
        lcc_model, mshares, price, tot_price = {}, {}, {}, {}
        
        for fueldict in self.sfuels:
            # loop through each fuel, fueldict is entire subtree below each type of fuel
            fueltype = fueldict[keyfuel]                          
            lcc_model[fueltype], mshares[fueltype], price[fueltype] = {}, {}, {}
            for prod, vals in fueldict[keyprod].items():               
                lcc_model[fueltype][prod] = self.life_cycle_cost(last_price=vals[keyprice], cap_cost=vals[keycapcost])
                mshares[fueltype][prod] = self.market_share(lccs[fueltype], prod, v=v)                
                price[fueltype][prod] = self.get_price(mshares[fueltype][prod], lcc_model[fueltype][prod])    # (total_quants["electricity"])   
            tot_price[fueltype] = sum(price[fueltype][p]/len(fueldict[keyprod].keys()) for p in fueldict[keyprod].keys())
            # TODO!! check averaging method
            
        if return_all_info:
            return tot_price, price, mshares 
        else:
            return tot_price
            
            

In [40]:
class SupplyDemand:
    '''
    SUPPLY-DEMAND MODEL:
        - Demand: estimate Quantity
        - Supply: estimate Price
        - Repeat using updated prices    
    '''    
    def __init__(self, data, verbose=True,
                             demandkey="demand_step",
                             supplykey="supply_step",
                             keyfuel="kind"):
        self.data = data   
        self.techs = data[demandkey]
        self.fuels = data[supplykey]
        self.tempdata = deepcopy(data) # check in case data is unsafe to change
        
        # may want to make keys editable
        self.reskeys = ("dem_price_by_fuel", "quant_by_fuel", "sup_price_by_fuel")
        infovals = {}
        
        self.results = dict.fromkeys(self.reskeys, infovals)
        
        self.info = {}
        self.info["fuels"] = [ft[keyfuel] for ft in self.fuels]
        self.info["price_diff"] = []
        
        byfuel = []
        fuel_results = dict.fromkeys(self.info["fuels"], byfuel)        
        
        for restype, vals in self.results.items():
            self.results[restype] = fuel_results
        print(self.results)
        print("**")

        
        self.initialize()

        
    def initialize(self, keyfuel="kind"):
        
        self.demand = Demand(self.data)
        quant_hat = self.demand.fuel_demand       

        init_price = {ft[keyfuel]: find_price_for_fuel( fuel_type = ft[keyfuel], data = self.techs ) for ft in self.fuels}
        self.supply = Supply(self.data)
        price_hat = self.supply.tot_price
        new_res = [init_price, quant_hat, price_hat]

        for num, res in enumerate(new_res):
            for res_key, res_value in res.items():
                self.results[self.reskeys[num]][res_key] = [res_value]
        
 
            
    def iterate(self, relative_tol=0.05, max_iter=10, n_iter=0):
        price_diff, price_thres, ge_thres = {}, {}, {}
        for f in self.info["fuels"]:
            last_price = self.results["dem_price_by_fuel"][f][-1]    
            cur_price = self.results["sup_price_by_fuel"][f][-1]
            price_diff[f] = np.abs(last_price - cur_price)
            price_thres[f] = cur_price * relative_tol
            ge_thres[f] = price_diff[f] > price_thres[f]

        self.info["price_diff"].append( price_diff )
        
        while any(ge_thres) and n_iter < max_iter:
            for fuel, is_ge in ge_thres.items():
                new_price = self.results["sup_price_by_fuel"][fuel][-1]
                # change price for fuel belonging to all techs in initial `dataset`
                change_price_for_fuel( self.data["demand_step"], fuel, new_price )

            
            self.demand = Demand(self.data)
#             dem_mshares = self.demand.tech_mshares
            quant_hat = self.demand.fuel_demand
            new_price = {ft["kind"]: find_price_for_fuel( fuel_type = ft["kind"], data = self.techs ) for ft in self.fuels}
            self.supply = Supply(self.data)
            
            price_hat = self.supply.tot_price

            for ft, val in new_price.items():
                self.results["dem_price_by_fuel"][ft].append(val)
            for ft, val in quant_hat.items():    
                self.results["quant_by_fuel"][ft].append(val)
            for ft, val in price_hat.items():
                self.results["sup_price_by_fuel"][ft].append(val)          
                
    
            for f in self.info["fuels"]:
                last_price = self.results["dem_price_by_fuel"][f][-1]    
                cur_price = self.results["sup_price_by_fuel"][f][-1]
                price_diff[f] = np.abs(last_price - cur_price)
                price_thres[f] = cur_price * relative_tol
                ge_thres[f] = price_diff[f] > price_thres[f]
                
            self.info["price_diff"].append( price_diff )
            n_iter += 1



In [48]:
sd = SupplyDemand(data)

{'dem_price_by_fuel': {'electricity': [], 'gas': []}, 'quant_by_fuel': {'electricity': [], 'gas': []}, 'sup_price_by_fuel': {'electricity': [], 'gas': []}}
**


In [49]:
sd.results

{'dem_price_by_fuel': {'electricity': [31.42768280241502],
  'gas': [61.06622728811647]},
 'quant_by_fuel': {'electricity': [31.42768280241502],
  'gas': [61.06622728811647]},
 'sup_price_by_fuel': {'electricity': [31.42768280241502],
  'gas': [61.06622728811647]}}

In [43]:
sd.iterate()

In [ ]:
# class SupplyDemand:
#     '''
#     SUPPLY-DEMAND MODEL:
#         - Demand: estimate Quantity
#         - Supply: estimate Price
#         - Repeat using updated prices    
#     '''    
#     def __init__(self, data, verbose=True,
#                              demandkey="demand_step",
#                              supplykey="supply_step",
#                              keyfuel="kind"):
#         self.data = data   
#         self.techs = data[demandkey]
#         self.fuels = data[supplykey]
#         self.tempdata = deepcopy(data) # check in case data is unsafe to change
        
#         # may want to make keys editable
#         self.reskeys = ( "dem_price_by_fuel", "quant_by_fuel", "sup_price_by_fuel")
#         self.infokeys = ("price_diff", "quantity_per_tech")
#         infovals = []
        
#         self.results = dict.fromkeys(self.reskeys, infovals)
#         self.info = dict.fromkeys(self.infokeys, infovals)
#         self.info["fuels"] = [ft[keyfuel] for ft in self.fuels]

#         # Not necessary to store here
#         self.info["mshares_demand"] = []
        
#         self.initialize()

        
#     def initialize(self, keyfuel="kind"):
        
#         self.demand = Demand(self.data)
#         dem_mshares = self.demand.tech_mshares
#         quant_hat = self.demand.tech_demand
#         init_price = {ft[keyfuel]: find_price_for_fuel( fuel_type = ft[keyfuel], data = self.techs ) for ft in self.fuels}
#         self.supply = Supply(self.data)
#         price_hat = self.supply.tot_price
#         new_res = [init_price, quant_hat, price_hat]

#         for idx, name in enumerate(self.reskeys):       
#             self.results[name].append(new_res[idx])

            
#         self.info["mshares_demand"].append(dem_mshares)
        
        
          
#     def iterate(self, relative_tol=0.05, max_iter=10, n_iter=0):
#         price_diff, price_thres, ge_thres = {}, {}, {}
#         for f in self.info["fuels"]:
#             last_price = self.results["dem_price_by_fuel"][-1][f]    
#             cur_price = self.results["sup_price_by_fuel"][-1][f]
#             price_diff[f] = np.abs(last_price - cur_price)
#             price_thres[f] = cur_price * relative_tol
#             ge_thres[f] = price_diff[f] > price_thres[f]

#         self.info["price_diff"].append( price_diff )
        
#         while any(ge_thres) and n_iter < max_iter:
#             for fuel, is_ge in ge_thres.items():
#                 new_price = self.results["sup_price_by_fuel"][-1][fuel]
#                 # change price for fuel belonging to all techs in initial `dataset`
#                 change_price_for_fuel( self.data["demand_step"], fuel, new_price )

            
#             self.demand = Demand(self.data)
#             dem_mshares = self.demand.tech_mshares
#             quant_hat = self.demand.fuel_demand
#             new_price = {ft["kind"]: find_price_for_fuel( fuel_type = ft["kind"], data = self.techs ) for ft in self.fuels}
#             self.supply = Supply(self.data)
            
#             price_hat = self.supply.tot_price

#             self.results["dem_price_by_fuel"].append(new_price)
#             self.results["quant_by_fuel"].append(quant_hat)
#             self.results["sup_price_by_fuel"].append(price_hat)
#             self.info["mshares_demand"].append(dem_mshares)             
                
    
#             for f in self.info["fuels"]:
#                 last_price = self.results["dem_price_by_fuel"][-1][f]    
#                 cur_price = self.results["sup_price_by_fuel"][-1][f]
#                 price_diff[f] = np.abs(last_price - cur_price)
#                 price_thres[f] = cur_price * relative_tol
#                 ge_thres[f] = price_diff[f] > price_thres[f]
                
#             self.info["price_diff"].append( price_diff )
#             n_iter += 1

